# Creando el modelo de regresión

En este apartado cargaremos los datos históricos y crearemos un modelo de regresión para hacer los cálculos de predicción. Para este ejemplo, utilizaremos el modelo SVR (`Support Vector Regression` del módulo de [scikit-learn](https://scikit-learn.org/stable/index.html) de [Python](https://www.python.org/): [sklearn.svm.SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)), pero se podrían utilizar otros modelos de regresión para comparar los resultados.

Los pasos que se seguirán en este *notebook* son los siguientes:

 1. [Carga de datos históricos](#Carga-de-datos-históricos)
 2. [Preparación de los datos](#Preparación-de-los-datos)
 3. [Normalización](#Normalización)
 4. [Añadiendo información a los datos de entrada](#Añadiendo-información-a-los-datos-de-entrada)
 5. [Training/Test/Validation Set](#Training/Test/Validation-Set)
 6. [Creando el modelo](#Creando-el-modelo)
 7. [Visualizando los resultados](#Visualizando-los-resultados)

## Carga de datos históricos

Vamos a empezar cargando los datos:

In [ ]:
import pandas as pd

df = pd.read_csv('../Datos/Datos2018.txt', parse_dates=['Fecha'])

df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

Nuestros `dataframe` de momento sólo contiene 2 columnas:

 - `Fecha`: día y hora de la emisión del ticket (formato `%Y-%m-%d %H:%M:%S`)
 - `Importe`: importe del ticket en €

In [ ]:
df.tail()

y el tipo de dato que contiene cada columna:
 - `Fecha`: tipo de dato `datetime`
 - `Importe`: tipo de dato `float`

In [ ]:
df.info()

## Preparación de los datos

Como se ha indicado al inicio, el objetivo es crear un modelo de **regresión**, por lo que tenemos que preparar los datos de forma que tengamos unos datos de entrada $X$ y otros de salida $Y$.

Para ello, vamos a intentar transformar nuestro `dataframe` para que tenga un formato similar a la siguiente tabla (en donde las ventas se van acumulando cada 15 minutos hasta llegar a las ventas totales del día: `22:00`):

| Dia | 09:00 | 09:15 | 09:30 | ... | 21:30 | 21:45 | 22:00 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 2019-09-26 | 0 | 20 000 | 51 000 | ... | 5 000 000 | 5 000 100 | 5 000 150 |
| 2019-09-27 | 0 | 20 200 | 51 400 | ... | 5 500 000 | 5 500 100 | 5 500 150 |
| 2019-09-28 | 0 | 23 000 | 53 000 | ... | 6 000 000 | 6 000 100 | 6 000 150 |

De esta manera, tendríamos los datos de entrada $X$ (en el que se introducirán más variables):

| 09:00 | 09:15 | 09:30 | ... | 21:30 | 21:45 |
| --- | --- | --- | --- | --- | --- |
| 0 | 20 000 | 51 000 | ... | 5 000 000 | 5 000 100 |
| 0 | 20 200 | 51 400 | ... | 5 500 000 | 5 500 100 |
| 0 | 23 000 | 53 000 | ... | 6 000 000 | 6 000 100 |

Y los datos de salida $Y$:

| 22:00 |
| --- |
| 5 000 150 |
| 5 500 150 |
| 6 000 150 |

Para preparar esa tabla, vamos a empezar agregando las ventas en intervalos de 15 minutos:

In [ ]:
df.index = df.pop('Fecha')
df = df.resample('15T').sum()

df.tail()

Continuaremos desglosando la `Fecha` en 2 nuevas columnas:
 - `Dia`
 - `Hora`

In [ ]:
df['Dia'] = df.index.map(lambda x: x.strftime('%Y-%m-%d'))
df['Hora'] = df.index.map(lambda x: x.strftime('%H:%M'))

df.tail()

Ahora ya estamos listos para formatear nuestro `dataframe`, para ello utilizaremos la función [crosstab](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) del módulo de [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html):

In [ ]:
df = pd.crosstab(index=df['Dia'],
                 columns=[df['Hora']],
                 values=df.Importe,
                 aggfunc=sum).fillna(0).reset_index()

df.set_index('Dia', inplace=True)

df.tail()

Para finalizar con esta parte, vamos hacer la suma acumulada de cada fila:

In [ ]:
df = df.cumsum(axis=1)

df.tail()

## Normalización

Para la normalización de los datos vamos a dividir todos los campos por el máximo valor de la tabla. De esa manera, todos nuestros valores oscilarán entre 0 y 1.

In [ ]:
max_value = df['23:45'].max()
df /= max_value
df.reset_index(inplace=True)

df.tail()

## Añadiendo información a los datos de entrada

Una vez que hemos preparado los datos para poder aplicar un modelo de regresión, vamos a añadir información adicional a los datos para intentar crear un modelo más afinado. En este caso vamos a añadir información de calendario:

 - Día de la semana
 - Días festivos
 
Esas nuevas variables van a ser variables **categóricas** por lo que crearemos [variables *dummy*](https://medium.com/hugo-ferreiras-blog/dealing-with-categorical-features-in-machine-learning-1bb70f07262d) para introducirlo en el modelo.

### Días de la semana

In [ ]:
df['Dia'] = pd.to_datetime(df['Dia'])
weekdays = [
    [0, 'Lunes'],
    [1, 'Martes'],
    [2, 'Miercoles'],
    [3, 'Jueves'],
    [4, 'Viernes'],
    [5, 'Sabado'],
    [6, 'Domingo']
]
for weekday, weekday_name in weekdays:
    df[weekday_name] = df['Dia'].map(lambda x: x.weekday() == weekday)
    
df[['Dia', 'Lunes', 'Martes', 'Miercoles',
    'Jueves', 'Viernes', 'Sabado', 'Domingo']].tail()

### Días Festivos

In [ ]:
calendario = ['2019-01-01']

df['Festivo'] = df['Dia'].isin(calendario)

## Training/Test/Validation Set

Por último, antes de crear el modelo vamos a dividir los datos en 3 bloques:

 - **Training Set**: 80% de los datos
 - **Test Set**: 10% de los datos
 - **Validation Set**: 10% de los datos

In [ ]:
def training_test_set():
    # Training Set (80%)
    train_data = df.sample(frac=0.8, random_state=0)
    test_validation_data = df.drop(train_data.index)

    # Test/Validatin Set (10%/10%)
    test_data = test_validation_data.sample(frac=0.5, random_state=0)
    validation_data = test_validation_data.drop(test_data.index)

    # Definir la variable 'Y'
    train_y = train_data.pop('23:45')
    test_y = test_data.pop('23:45')
    validation_y = validation_data.pop('23:45')

    # Eliminamos la columna 'Dia'
    train_data.drop(['Dia'], axis=1, inplace=True)
    test_data.drop(['Dia'], axis=1, inplace=True)
    validation_data.drop(['Dia'], axis=1, inplace=True)
    
    return [train_data, test_data, validation_data, train_y, test_y, validation_y]

## Creando el modelo

Como se ha indicado en la introducción, en este ejemplo se va a utilizar el modelo [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) para la regresión y se va a elegir el `kernel` `rbf`.

No es objeto de este *notebook* explicar los detalles de este modelo, pero el lector que esté interesado en profundizar en este modelo puede ver con más detalle los algoritmos que se utilizan para hacer los cálculos en la documentación oficial de [scikit-learn](https://scikit-learn.org/stable/modules/svm.html#svm-regression).

En `SVR` hay básicamente 2 parámetros que se utilizan para ajustar el modelo:

 - `C`
 - `Epsilon`
 
a los que vamos a darle diferentes valores.

In [ ]:
from sklearn.svm import SVR

def crear_modelo():
    error = -1

    for C in [0.1, 1, 100, 1000]:
        for epsilon in [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10]:
            # Creamos el modelo con los parámetros seleccionados
            svr_rbf = SVR(kernel='rbf', C=C, gamma='auto', epsilon=epsilon)

            # Ajustamos el modelo a nuestros datos
            model = svr_rbf.fit(train_data, train_y)

            # Medir la calidad del modelo con el Test Set
            error_now = (model.predict(test_data) - test_y).std()

            # Guardar los parámetros si se ha mejorado el error
            if (error_now < error) or (error == -1):
                error = error_now
                C_good = C
                epsilon_good = epsilon
                
    return SVR(kernel='rbf', C=C_good, gamma='auto', epsilon=epsilon_good)

Como queremos crear diferentes modelos para cada hora, a la hora de crear el modelo tendremos que modificar las variables de entrada $X$, eliminando las columnas que no se van a tener en cuenta para el modelo. Para ese fin vamos a definir la siguiente función:

In [ ]:
def eliminar_columnas(hour_now):
    """Eliminar las columnas que no se van a utilizar para el cálculo de las previsiones"""

    cols_drop = df.columns[(df.columns > hour_now) & (df.columns < '23:45')]
    for col in cols_drop:
        df.pop(col)

También vamos a necesitar otra función para guardar los modelos utilizando el módulo `joblib`

In [ ]:
import os
import joblib


def guardar_modelo_svr(name):
    """Guardar los resultados del modelo"""

    folder = 'modelos/'
    if not os.path.exists(folder):
        os.makedirs(folder)

    filename = 'modelo_' + name + '.sav'
    joblib.dump(model, folder + filename)

In [ ]:
df_orig = df.copy()

Pongamos en marcha los cálculos...

In [ ]:
for hour in range(10, 22):
    print('\nCreando modelo para las ' + str(hour) + '...')
    df = df_orig.copy()
    eliminar_columnas(str(hour) + ':00')
    train_data, test_data, validation_data, train_y, test_y, validation_y = training_test_set()
    model = crear_modelo()
    model.fit(train_data, train_y)
    guardar_modelo_svr(str(hour) + '00')
    print('Modelo guardado!')
    print('Error de validación: ' + str((model.predict(validation_data) - validation_y).std()))

# Guardamos también el valor máximo 'max_value'
file = open('modelos/max_value.txt', 'w')
file.write(str(round(max_value, 2)))
file.close()

Si todo ha ido bien, habremos creado los siguientes archivos en nuestro ordenador (puede variar ligeramente la imagen), es decir, habremos guardado con éxito los modelos.

```
1. Creando el modelo predictivo/
    Datos
    Modelo/
        .ipynb_checkpoints
        modelos/
            max_value.txt
            modelo_1000.sav
            modelo_1100.sav
            modelo_1200.sav
            modelo_1300.sav
            modelo_1400.sav
            modelo_1500.sav
            modelo_1600.sav
            modelo_1700.sav
            modelo_1800.sav
            modelo_1900.sav
            modelo_2000.sav
            modelo_2100.sav
            modelo_2200.sav
            modelo_2300.sav
    Modelo.ipynb
```

## Visualizando los resultados

Cargamos las librerias de [Plotly](https://plot.ly/python/)

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

Seleccionamos los días que queremos mostrar

In [ ]:
dias = df_orig.loc[df_orig.index.isin(validation_y.index), 'Dia']

Cargamos el modelo que queremos analizar:

In [ ]:
hour = 14

# Cargamos el modelo
loaded_model = joblib.load('modelos/modelo_{}00.sav'.format(str(hour)))

# Creamos los datos de validación
df = df_orig.copy()
eliminar_columnas(str(hour) + ':00')
train_data, test_data, validation_data, train_y, test_y, validation_y = training_test_set()

Visualizamos

In [ ]:
iplot({
    'data': [go.Scatter(
                x=dias,
                y=validation_y * max_value,
                name='Real'
            ),go.Scatter(
                x=dias,
                y=loaded_model.predict(validation_data) * max_value,
                name='Predicción'
    )],
    'layout': go.Layout(
                yaxis={
                    'title': 'Ventas €'
                }
    )
})